# State

## O que é?

O padrao _State_, é um _behavioral pattern_ baseado na ideia de formalizar o estado de objetos como uma familia de classes, os _States_, e utilizá-las para abstrair operações que mudem de acordo com o estado.

## Por quê?

Muitas vezes o comportamento de um objeto muda no _runtime_ de acordo com seu estado interno. Para implementar este comportamento da forma convencional, o desenvolver armazenaria o estado interno como atributos simples e utilizaria `if` ou `switch` _statements_ para que o objeto se comportasse de acordo com seu estado atual.

Essa implementação pode tornar o código muito complexo, difícil de ser estendido e pode resultar em estados inválidos. Ao encapsular os estado interno em um classe, a adição de novos estados torna-se simples e as transições mais fáceis de serem compreendidas.

## Uso:

Você deve usar o _State_ quando:
- Um classe tem estados internos bem definidos e que modificam seu comportamento.
- O estado do objeto evolue dinamicamente durante o _run time_ .

## Estrutura: 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/State_Design_Pattern_UML_Class_Diagram.svg/1200px-State_Design_Pattern_UML_Class_Diagram.svg.png" alt="State Design Pattern UML Class Diagram.svg">

## Exemplo:

Vamos escrever uma class usada construir _strings_ .

In [2]:
blah = "a"

In [10]:
class StringBuilder:
    """
    Writes a string bit by bit.
    """
    def __init__(self, mode="w"):
        self.mode = mode
        self.locked = False
        self.output = ""
        
    def write(self, string):
        """Append to the string"""
        if self.locked:
            raise Exception("Builder must be unlocked in order to write.")
        if self.mode == "w":
            self.output += string
        else:
            self.output += string.decode()
            
    def read(self):
        """Read the final string."""
        if not self.locked:
            raise Exception("Builder must be locked in order to read.")
        if self.mode == "w":
            return self.output
        else:
            return self.output.encode()
    
    def lock(self):
        """Lock the builder and enables reads."""
        self.locked = True

In [11]:
builder = StringBuilder()
builder.write("isso")
builder.write(" eh")
builder.write(" um test.")

In [12]:
builder.read()

Exception: Builder must be locked in order to read.

In [13]:
builder.lock()
builder.read()

'isso eh um test.'

Isso funciona bem, porém decidimos que em certos casos, ter que travar o _builder_ para ler a _string_ é problematico, então resolvemos introduzir os modos `rw` e `rwb`.

In [ ]:
class StringBuilder:
    """
    Writes a string bit by bit.
    """
    def __init__(self, mode="w"):
        self.mode = mode
        self.locked = False
        self.output = ""
        
    def write(self, string):
        """Append to the string"""
        if self.locked:
            raise Exception("Builder must be unlocked in order to write.")
        if self.mode in ("w", "rw"):  # mudou
            self.output += string
        else:
            self.output += string.decode()
            
    def read(self):
        """Read the final string."""
        if not self.locked and self.mode not in ("rw", "rwb"):  # mudou
            raise Exception("Builder must be locked in order to read.")
        if self.mode in ("w", "rw"):  # mudou
            return self.output
        else:
            return self.output.encode()
    
    def lock(self):
        """Lock the builder and enables reads."""
        self.locked = True

Isso pode se tornar código espagate rapidamente.

Tentemos usar o padrão _State_.

In [23]:
import abc

class StringBuilder:
    """
    Writes a string bit by bit.
    """
    
    def __init__(self, mode="w"):
        state_map = {
            "w": Write,
            "wb": WriteBytes,
            "rw": ReadWrite,
            "rwb": ReadWriteBytes,
        }
        
        self.mode = state_map[mode]
        self.output = ""
        
    def write(self, string):
        self.mode.write(self, string)
        
    def read(self):
        return self.mode.read(self)
    
    def lock(self):
        return self.mode.lock(self)
    
    
class Mode(metaclass=abc.ABCMeta):
    """
    Current state of the builder, it takes care of state specific
    operations and state transations.
    """
    @staticmethod
    def write(builder, string):
        raise NotImplementedError("Invalid operation.")
    
    @staticmethod
    def read(builder):
        raise NotImplementedError("Invalid operation.")
        
    @staticmethod
    def lock(builder):
        raise NotImplementedError("Invalid operation.")
    
    
class Str(Mode):
    @staticmethod
    def lock(builder):
        builder.mode = Locked

        
class Bytes(Mode):
    @staticmethod
    def lock(builder):
        builder.mode = LockedBytes
        

class Write(Str):
    @staticmethod
    def write(builder, string):
        builder.output += string


class ReadWrite(Str):
    @staticmethod
    def write(builder, string):
        builder.output += string

    @staticmethod
    def read(builder):
        return builder.output
    

class Locked(Str):
    @staticmethod
    def read(builder):
        return builder.output  

        
class WriteBytes(Bytes):
    @staticmethod
    def write(builder, string):
        builder.output += string.decode()


class ReadWriteBytes(Bytes):
    @staticmethod
    def write(builder, string):
        builder.output += string.decode()

    @staticmethod
    def read(builder):
        return builder.output.encode()
    

class LockedBytes(Bytes):
    @staticmethod
    def read(builder):
        return builder.output.encode() 

In [24]:
builder = StringBuilder()
builder.write("isso")
builder.write(" eh")
builder.write(" um test.")

In [25]:
builder.read()

NotImplementedError: Invalid operation.

In [26]:
builder.lock()
builder.read()

'isso eh um test.'

É... não vale a pena...

## Prós e contras:

### Prós

- Reduz o número de `if` _statements_ , melhorando a legibilidade e reduzindo a complexidade do código.
- Os estados tornam-se discretos, evitando que o objeto atinja um estado inválido.
- Os _states_ são opacos para o _context_, tornando simples a inclusão de novos _states_.
- Transição entre estados torna-se mais simples.

### Contras

- O número de classes tende a aumentar significativamente.
- Esse padrão torna difícil a reutilização de código já que cada estado tem sua própria classe.

## Discussao:

1. If something has only two to three states, is it overkill to use the State pattern?

2. Como esse padrão se compara com o padrão _Strategy_ ? E o padrão _Abstract Factory_ ?